# Python - Getting more information from Tracebacks
> A tutorial to get more information from Python exception stack traceback.

- toc: true 
- badges: true
- comments: true
- categories: [python]

## About
This notebook demonstrates what Python Traceback object is, and how can we get more information out of it to better diagnose exception messages.

### Credit
This blog post is adapted from an article that was originally written in `Python Cookbook` published by `O'Reilly Media, Inc.` and released July 2002. In this book under chapter 15 there is a section with title `Getting More Information from Tracebacks` written by `Bryn Keller`. An online version of this article can be found at https://www.oreilly.com/library/view/python-cookbook/0596001673/ch14s05.html.

Original article uses Python 2.2, but I have adapted it for Python 3.8. Also, I have added some commentory to give more insights on Python Traceback object.

### Environment Details 

In [1]:
#collapse-hide
from platform import python_version

print("python==" + python_version())

python==3.9.7


## Discussion

Consider a following toy example where we are getting some data from an external source (could be an API call, or a DB call), and we need to find the length of individual items provided in the list. We know that items in the list will be of type `str` so we have used a `len()` function on it.

Once we ran our function on received data we got an exception, and now we are trying to investigate what caused the error.

In [2]:
#collapse-hide
# this is intentionally hidden as we don't know about the data received from external source. 
data = ["1", "22", 333, "4444"]

In [3]:
##
# our toy example function.
import sys, traceback

def get_items_len(items: list) -> list:
    """
    this function returns the length of items received in a list.
    """
    items_len = []
    for i in items:
        items_len.append(len(i))
    
    return items_len

In [4]:
##
# lets run our function on "data" received from an external source
try:
    get_items_len(data)
except Exception as e:
    print(traceback.print_exc())

None


Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/4026001270.py", line 4, in <module>
    get_items_len(data)
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/3663053895.py", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()


This is not looking good. While processing the data we have got an exception. The `Traceback` message is giving us some details. It tells us that we have received some data of type _integer_ instead of _string_, and we are trying to call _len()_ function on it. But we don't know the actual data value that caused the exception, or we don't know the _index_ of the item in the list that caused this error. Depending on the use case, information about the local variables state, or data that actually caused the error can be crutial in diagnosing the root cause of an  error.

Fortunately, all this information is already available to us in 'Traceback' object, but there is no builtin method that can give this information directly. Let us try some of the builtin methods on Traceback object to see the kind of information we could get from it.

In [5]:
#collapse-output
# calling traceback module builtin methods
try:
    get_items_len(data)
except Exception as e:
    print("***** Exception *****")
    print(e)

    exc_type, exc_value, exc_traceback = sys.exc_info()
    print("\n***** print_tb *****")
    traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)

    print("\n***** print_exception *****")
    # exc_type below is ignored on 3.5 and later
    traceback.print_exception(exc_type, exc_value, exc_traceback,
                                limit=2, file=sys.stdout)
    print("\n***** print_exc *****")
    traceback.print_exc(limit=2, file=sys.stdout)

    print("\n***** format_exc, first and last line *****")
    formatted_lines = traceback.format_exc().splitlines()
    print(formatted_lines[0])
    print(formatted_lines[-1])

    print("\n***** format_exception *****")
    # exc_type below is ignored on 3.5 and later
    print(repr(traceback.format_exception(exc_type, exc_value,
                                            exc_traceback)))
                                            
    print("\n***** extract_tb *****")
    print(repr(traceback.extract_tb(exc_traceback)))

    print("\n***** format_tb *****")
    print(repr(traceback.format_tb(exc_traceback)))

    print("\n***** tb_lineno *****", exc_traceback.tb_lineno)

***** Exception *****
object of type 'int' has no len()

***** print_tb *****
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/1947054714.py", line 4, in <module>
    get_items_len(data)

***** print_exception *****
Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/1947054714.py", line 4, in <module>
    get_items_len(data)
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/3663053895.py", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()

***** print_exc *****
Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/1947054714.py", line 4, in <module>
    get_items_len(data)
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/3663053895.py", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()

***** format_exc, first and last line *****
Traceback (most recent call last):
TypeError: object of type 'int' h

All these methods are useful but we are still short on information about the state of local variables when the system crashed.

Before writing our custom function to get variable state, let us spend some time to understand the working of Python Traceback object.

### Traceback Module
> https://docs.python.org/3/library/traceback.html

This module provides an easy to use interface to work with `traceback objects`. It provides multiple functions that we can use extract required information from traceback. So far we have used methods from this module in above examples.


### Traceback Objects
> https://docs.python.org/3/reference/datamodel.html <br>
> On this page search for term "Traceback objects"

Traceback objects represent a stack trace of an exception. A traceback object is implicitly created when an exception occurs, and may also be explicitly created by initializing an instance of class `types.TracebackType`. _traceback_ object is also an instance of _types.TracebackType_ class. When an exception occurs traceback object is initialized for us, and we can obtain it from any of the following two methods. 
1. It is available as a third item of the tuple returned by sys.exc_info() "`(type, value, traceback)`"
2. It is available as the `__traceback__` object of the caught exception. "`Exception.__traceback__`"

A traceback object is basically a linked list of nodes, where each node is a `Frame object`. Frame objects form their own linked list but in opposite direction of traceback objects. Together they work like a doubly linked list, and we can use them to move back and forth in the stack trace history. It is the frame objects that hold all the stack important information. traceback object has some special attributes
* `tb_next` point to the next level in the stack trace (towards the frame where the exception occurred), or None if there is no next levela
* `tb_frame` points to the execution frame of the current level
* `tb_lineno` gives the line number where the exception occurred

In [6]:
##
# method 1: get traceback object using sys.exc_info()
try:
    get_items_len(data)
except Exception as e:
    print(sys.exc_info()[2])

In [7]:
##
# method 2: get traceback object using Exception.__traceback__
try:
    get_items_len(data)
except Exception as e:
    print(e.__traceback__ )

If there is no exception in the system, then calling sys.exc_info() will only return `None` values.

In [8]:
##
# no exception generated so sys.exc_info() will return None values.
try:
    get_items_len(['1','2','3','4'])
except Exception as e:
    print(sys.exc_info()[2])

### Frame Objects
> https://docs.python.org/3/reference/datamodel.html <br>
> On this page search for term "Frame objects"

Frame objects represents execution frames. It has some special attributes
* `f_back` It is reference to the previous stack frame (towards the caller), or None if this is the bottom stack frame
* `f_code` It is the code object being executed in this frame. We will discuss `Code Objects` in next section
* `f_lineno` It is the current line number of the frame — writing to this from within a trace function jumps to the given line (only for the bottom-most frame). A debugger can implement a Jump command (aka Set Next Statement) by writing to f_lineno. This attribute will give you the line number in code at which exception occured
* `f_locals` A dictionary used to lookup local variables. From this dictionary we can get all the local variables and their state at the time of exception
* `f_globals` A dictionary for global varaibles

### Code Objects
> https://docs.python.org/3/reference/datamodel.html <br>
> On this page search for term "Code Objects"

Code objects represent byte-compiled executable Python code, or bytecode. It has some very helpful attributes
* `co_name` It gives the function name being executed
* `co_filename` It gives the filename from which the code was compiled

There are many other very special attributes in this object but we are not interested in them for now. You may read about them from the docs.

### Visual Representation of Traceback, Frame and Code Objects
![](..\images\2022\2022-02-11-python-traceback.png)

### Custom fuction for additional exception info
Now with some addition information about stack trace internal objects, lets create a function to get variables state at the time of exception.

In [9]:
#collapse-show
def exc_info_plus():
    """
    Provides the usual traceback information, followed by a listing of all the
    local variables in each frame.
    """
    tb = sys.exc_info()[2]

    # iterate forward to the last (most recent) traceback object.
    while 1:
        if not tb.tb_next:
            break
        tb = tb.tb_next
    stack = []

    # get the traceback frame
    f = tb.tb_frame

    # iterate backwards from recent to oldest traceback frame 
    while f:
        stack.append(f)
        f = f.f_back
    
    # oldest frame is at the top. reverse it to get the recent frame at the top.
    stack.reverse()

    # get exception information and stack trace entries from most recent traceback object
    exc_msg = traceback.format_exc()

    exc_msg += "\n*** Locals by frame, innermost last ***"
    for frame in stack:
        exc_msg += f"\nFrame {frame.f_code.co_name} in {frame.f_code.co_filename} at line {frame.f_lineno}"
        for key, value in frame.f_locals.items():
            exc_msg += f"\n\t {key:20} = "
            try:
                data = str(value)
                # limit variable output to a certain number. But you can adjust it as per your requirement.
                # Better to have an upper limit. If removed then large objects (e.g. Pandas DataFrame) can be troublesome. 
                output_limit = 50
                exc_msg += (data[:output_limit] + "...") if len(data) > output_limit else data
            except:
                exc_msg += "<ERROR WHILE PRINTING VALUE>"

    return exc_msg

In [10]:
#collapse-output
#now lets try our custom exception function
try:
    get_items_len(data)
except Exception as e:
    print(exc_info_plus())

Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/1949264869.py", line 4, in <module>
    get_items_len(data)
  File "C:\Users\HP\AppData\Local\Temp/ipykernel_13208/3663053895.py", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()

*** Locals by frame, innermost last ***
Frame _run_module_as_main in C:\Users\HP\anaconda3\lib\runpy.py at line 197
	 mod_name             = ipykernel_launcher
	 alter_argv           = True
	 mod_spec             = ModuleSpec(name='ipykernel_launcher', loader=<_fro...
	 code                 = <code object <module> at 0x0000018389A6CDF0, file ...
	 main_globals         = {'__name__': '__main__', '__doc__': 'Entry point f...
Frame _run_code in C:\Users\HP\anaconda3\lib\runpy.py at line 87
	 code                 = <code object <module> at 0x0000018389A6CDF0, file ...
	 run_globals          = {'__name__': '__main__', '__doc__': 'Entry point f...
	 init_globals         = Non

Note the last three lines in above stack trace. How easy it is to see the input (_items_) that we received in our function. Also the item at index _i_ is 333 is also available on which our function crashed. Using our custom function unexpected errors are logged in a format that makes it a lot easier to find and fix the errors. Finally, we can fix our function to handle unexpected integer values

In [11]:
##
# lets fix our function to handle unexpected 'int' items by converting them to 'str'
def get_items_len(items: list) -> list:
    """
    this function returns the length of items received in a list.
    """
    items_len = []
    for i in map(str, items):
        items_len.append(len(i))
    
    return items_len

# let's test our function again
get_items_len(data)

[1, 2, 3, 4]